In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

max_features = 10000
maxlen = 20
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_teset = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_train[0])

2025-04-21 19:16:06.002767: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
(25000, 20) (25000,)
(25000,) (25000,)
[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
   15   16 5345   19  178   32]


In [22]:
word_index = imdb.get_word_index()

index_to_word = {index + 3: word for word, index in word_index.items()}
#matching the Keras style of embedding
index_to_word[0]= "<PAD>"
index_to_word[1] = "<START>"
index_to_word[2] = "<UNK>"
index_to_word[3] = "<UNUSED>"

def decode_review(encoded_review):
    return ' '.join([index_to_word.get(i, '?') for i in encoded_review])

decoded_review = decode_review(x_train[0])
print(decoded_review)


story was so lovely because it was true and was someone's life after all that was shared with us all


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
print(x_train.shape)

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train, epochs =10, batch_size=32, validation_split=0.2)

(25000, 20)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.5704 - loss: 0.6852 - val_acc: 0.6982 - val_loss: 0.6153
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.7448 - loss: 0.5647 - val_acc: 0.7296 - val_loss: 0.5257
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.7876 - loss: 0.4663 - val_acc: 0.7448 - val_loss: 0.5029
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8137 - loss: 0.4199 - val_acc: 0.7524 - val_loss: 0.4982
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8281 - loss: 0.3910 - val_acc: 0.7544 - val_loss: 0.4988
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8385 - loss: 0.3736 - val_acc: 0.7522 - val_loss: 0.5025
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8473 - loss: 0.3557 - val_acc: 0.7512 - val_loss: 0.5066
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - acc: 0.8593 - loss: 0.3338 - val_acc: 0.7536 - val_loss: 0.5140
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - ac

In [28]:
embedding_layer = model.layers[0]
embedding_matrix = embedding_layer.get_weights()[0]
print("임베딩 행렬 모양:", embedding_matrix.shape)

word_index = imdb.get_word_index()
index_to_word = {i + 3: w for w, i in word_index.items()}
index_to_word[0] = "<PAD>"; index_to_word[1] = "<START>"
index_to_word[2] = "<UNK>"; index_to_word[3] = "<UNUSED>"

def show_vector(word):
    idx = word_index.get(word)
    if idx is None or idx + 3 >= embedding_matrix.shape[0]:
        print("단어가 어휘사전에 없습니다.")
        return
    vec = embedding_matrix[idx+3]
    print(f"'{word}' 임베딩:", vec)

show_vector("good")
show_vector("bad")
show_vector("movie")
show_vector("bomb")

임베딩 행렬 모양: (10000, 8)
'good' 임베딩: [-0.05726561  0.03438354 -0.21095692 -0.18333328 -0.19895193  0.0698124
  0.15729015  0.00870921]
'bad' 임베딩: [ 0.05152619  0.34251812  0.46294582  0.49180067  0.3271383  -0.18326822
 -0.31053972 -0.26370612]
'movie' 임베딩: [-0.03207814  0.07805683  0.02683133  0.0874258   0.010656    0.17133264
 -0.03301812  0.11544038]
'bomb' 임베딩: [ 0.10289402  0.35928193  0.36207235  0.3380123   0.41198432 -0.28671098
 -0.2687604  -0.13778044]


In [44]:
"""
new_reviews = [
    "It was a boring movie. I don't want to recommend it.",
    "The movie was extremely disappoing."
]"""

new_reviews = [
    "I absolutely loved this movie. The storyline was gripping and the characters were very well developed. The performances were outstanding and the direction was top-notch. The cinematography was beautiful and the soundtrack was perfect. This is one of the best movies I've seen in a long time and I would highly recommend it to everyone.",
    "This film was an incredible journey from start to finish. The visuals were stunning and the soundtrack perfectly complemented the emotional tone of the movie. The plot was engaging and the characters were relatable and well-acted. It left a lasting impression on me and I can’t wait to watch it again. A must-see for any movie lover!",
    "The movie was extremely disappointing. The plot was weak and the characters were not believable. The acting was subpar and the film dragged on far too long. I wouldn't recommend this movie.",
    "I found the film to be quite boring. The storyline was predictable and there was no character development. The acting was mediocre at best and the whole movie felt like a waste of time. Definitely not worth watching."
]

new_sequences = []
for review in new_reviews:
    sequence = [word_index.get(word.lower(), 2) for word in review.replace('.', ' ').split()]
    sequence = [index if index < max_features else 2 for index in sequence]
    new_sequences.append(sequence)


padded_new_sequences = preprocessing.sequence.pad_sequences(new_sequences, maxlen=maxlen)

predictions = model.predict(padded_new_sequences)

for review, prediction in zip(new_reviews, predictions):
    print(f"Review: {review} -> Prediction: {'Pos' if prediction > 0.5 else 'Neg'} (Percentage: {prediction[0]:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Review: I absolutely loved this movie. The storyline was gripping and the characters were very well developed. The performances were outstanding and the direction was top-notch. The cinematography was beautiful and the soundtrack was perfect. This is one of the best movies I've seen in a long time and I would highly recommend it to everyone. -> Prediction: Neg (Percentage: 0.2061)
Review: This film was an incredible journey from start to finish. The visuals were stunning and the soundtrack perfectly complemented the emotional tone of the movie. The plot was engaging and the characters were relatable and well-acted. It left a lasting impression on me and I can’t wait to watch it again. A must-see for any movie lover! -> Prediction: Pos (Percentage: 0.7189)
Review: The movie was extremely disappointing. The plot was weak and the characters were not believable. The acting was subpar and the film dragged on far too long. I wouldn't recommend this movie

In [70]:
print(new_sequences)
print(new_sequences[0].count(2))

[[10, 424, 444, 11, 17, 1, 766, 13, 3130, 2, 1, 102, 68, 52, 70, 1388, 1, 351, 68, 1336, 2, 1, 455, 13, 2, 1, 624, 13, 304, 2, 1, 813, 13, 401, 11, 6, 28, 4, 1, 115, 99, 204, 107, 8, 3, 193, 55, 2, 10, 59, 542, 383, 9, 5, 313], [11, 19, 13, 32, 1045, 1308, 36, 377, 5, 1360, 1, 2054, 68, 1377, 2, 1, 813, 947, 2, 1, 918, 1160, 4, 1, 17, 1, 111, 13, 1725, 2, 1, 102, 68, 2, 2, 2, 9, 314, 3, 5860, 1381, 20, 69, 2, 10, 2, 855, 5, 103, 9, 171, 3, 2, 15, 98, 17, 2], [1, 17, 13, 573, 1329, 1, 111, 13, 812, 2, 1, 102, 68, 21, 864, 1, 113, 13, 2, 2, 1, 19, 3314, 20, 227, 96, 193, 10, 583, 383, 11, 17], [10, 255, 1, 19, 5, 27, 176, 354, 1, 766, 13, 724, 2, 47, 13, 54, 106, 940, 1, 113, 13, 1498, 30, 115, 2, 1, 223, 17, 418, 37, 3, 434, 4, 55, 404, 21, 287, 146]]
5
